### Compare Listing 
<li>a: vector uniform</li>
<li>b: greedy</li>
<li>c: e - greedy</li>
<li>d: Linear Reward Penalty</li>
<li>e: Linear Reward Inaction</li>
<li>f: UBC</li>
<li>g: BayesianUCB</li>
<li>h: Thompson Sampling</li>
<li>i: Neural Network</li>
<li>j: Non Stationary</li>

In [39]:
# import lib
import numpy as np
import matplotlib.pyplot as plt
import scipy,time,sys
np.random.seed(5678)
np.set_printoptions(3)

In [61]:
# setting the ground truth
num_bandit = 12
num_ep  = 200
num_iter= 2000
gt_prob = np.random.uniform(0,1,num_bandit)
optimal_choice = np.argmax(gt_prob)
print(gt_prob)
print('Best Choice: ',optimal_choice,gt_prob[optimal_choice])

[0.77  0.29  0.795 0.553 0.671 0.403 0.445 0.47  0.225 0.195 0.567 0.347]
Best Choice:  2 0.7947818672937569


In [62]:
# vectorized
a_expect = np.zeros((num_ep,num_bandit))
                    
for eps in range(num_ep):
    temp_expect = np.zeros(num_bandit)
    temp_choice = np.zeros(num_bandit)
                    
    for iter in range(num_iter//10):
        temp_choice    = temp_choice + 1
        current_reward = np.random.uniform(0,1) < gt_prob
        temp_expect    = temp_expect + (1/(temp_choice+1)) * (current_reward - temp_expect)
    a_expect[eps,:] = temp_expect
                    
print('Ground Truth')
print(gt_prob)
print('Expected ')
print(a_expect.mean(0))

Ground Truth
[0.77  0.29  0.795 0.553 0.671 0.403 0.445 0.47  0.225 0.195 0.567 0.347]
Expected 
[0.762 0.285 0.788 0.546 0.664 0.397 0.44  0.464 0.221 0.193 0.559 0.342]


In [63]:
# greedy
b_pull_count   = np.zeros((num_ep,num_bandit))
b_estimation   = np.zeros((num_ep,num_bandit))
b_reward       = np.zeros((num_ep,num_iter))
b_optimal_pull = np.zeros((num_ep,num_iter))
                    
for eps in range(num_ep):
    temp_pull_count   = np.zeros(num_bandit)
    temp_estimation   = np.zeros(num_bandit)
    temp_reward       = np.zeros(num_iter)
    temp_optimal_pull = np.zeros(num_iter)
                    
    for iter in range(num_iter):
        
        # select bandit / get reward /increase count / update estimate
        current_choice = np.argmax(temp_expect)
        current_reward = 1 if np.random.uniform(0,1) < gt_prob[current_choice] else 0
        temp_pull_count[current_choice] = temp_pull_count[current_choice] + 1
        temp_estimation[current_choice] = temp_estimation[current_choice] + (1/(temp_pull_count[current_choice]+1)) * (current_reward-temp_estimation[current_choice])
        
        # update reward and optimal choice
        temp_reward[iter] = temp_reward[iter] + current_reward
        temp_optimal_pull[iter] = 1 if current_choice == optimal_choice else 0
        
    b_pull_count[eps,:]   = temp_pull_count
    b_estimation[eps,:]   = temp_estimation
    b_reward[eps,:]       = temp_reward
    b_optimal_pull[eps,:] = temp_optimal_pull
        
print('Ground Truth')
print(gt_prob)
print('Expected ')
print(b_estimation.mean(0))

Ground Truth
[0.77  0.29  0.795 0.553 0.671 0.403 0.445 0.47  0.225 0.195 0.567 0.347]
Expected 
[0.    0.    0.795 0.    0.    0.    0.    0.    0.    0.    0.    0.   ]


In [ ]:
# e greedy 
epsilon = np.random.uniform(0,1)
c_pull_count   = np.zeros((num_ep,num_bandit))
c_estimation   = np.zeros((num_ep,num_bandit))
c_reward       = np.zeros((num_ep,num_iter))
c_optimal_pull = np.zeros((num_ep,num_iter))
                    
for eps in range(num_ep):
    temp_pull_count   = np.zeros(num_bandit)
    temp_estimation   = np.zeros(num_bandit)
    temp_reward       = np.zeros(num_iter)
    temp_optimal_pull = np.zeros(num_iter)
                    
    for iter in range(num_iter):
        
        # select bandit / get reward /increase count / update estimate
        current_choice = np.argmax(temp_expect) if epsilon < np.random.uniform(0,1) else np.random.choice(np.arange(10))
        current_reward = 1 if np.random.uniform(0,1) < gt_prob[current_choice] else 0
        temp_pull_count[current_choice] = temp_pull_count[current_choice] + 1
        temp_estimation[current_choice] = temp_estimation[current_choice] + (1/(temp_pull_count[current_choice]+1)) * (current_reward-temp_estimation[current_choice])
        
        # update reward and optimal choice
        temp_reward[iter] = temp_reward[iter] + current_reward
        temp_optimal_pull[iter] = 1 if current_choice == optimal_choice else 0
        
    c_pull_count[eps,:]   = temp_pull_count
    c_estimation[eps,:]   = temp_estimation
    c_reward[eps,:]       = temp_reward
    c_optimal_pull[eps,:] = temp_optimal_pull
        
print('Ground Truth')
print(gt_prob)
print('Expected ')
print(c_estimation.mean(0))

# Reference 
1. numpy.set_printoptions — NumPy v1.14 Manual. (2019). Docs.scipy.org. Retrieved 13 January 2019, from https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.set_printoptions.html